In [11]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import math
from typing import Tuple, List

Parameters and function setting

In [3]:
def solution(x, y):
    return np.sin(3.0 * np.pi * x ** 2 * y) + (y - 1.0) ** 3

In [4]:
def f(x, y):
    return 18 * np.pi * (np.pi * x ** 2 * np.sin(3 * np.pi * x ** 2 * y) * (x ** 2 + 6 * y ** 2) - y * np.cos(3 * np.pi * x ** 2 * y)) - 12 * (y - 1)

In [5]:
def a(x, y):
    return 3

def b(x, y):
    return 2

Mapping function

In [6]:
def idx(i, j, N):
    return (j-1) * (N - 1) + (i-1)

SLAE compilation

In [7]:
def SLAE_compilation(N: int):
    h = 1.0 / N
    size = (N - 1) ** 2 
    
    A = np.zeros((size, size))
    rhs = np.zeros(size)
    
    coeff_x = a(0, 0) / h**2
    coeff_y = b(0, 0) / h**2
    
    for j in range(1, N):  # y
        for i in range(1, N):  # x
            k = idx(i, j, N=N)
            x, y = i*h, j*h
            
            # diagonal
            A[k, k] = 2*(a(0, 0)+b(0, 0))/h**2
            rhs[k] = f(x, y) 
            
            # neighbours
            # east (i+1,j)
            if i < N-1:
                A[k, idx(i+1,j, N)] = -coeff_x
            else:  # border x=1
                rhs[k] += coeff_x * solution(1.0, y)
            
            # west (i-1,j)
            if i > 1:
                A[k, idx(i-1,j, N)] = -coeff_x
            else:  # border x=0
                rhs[k] += coeff_x * solution(0.0, y)
            
            # nord (i,j+1)
            if j < N-1:
                A[k, idx(i,j+1, N)] = -coeff_y
            else:  # border y=1
                rhs[k] += coeff_y * solution(x, 1.0)
            
            # sud (i,j-1)
            if j > 1:
                A[k, idx(i,j-1, N)] = -coeff_y
            else:  # border y=0
                rhs[k] += coeff_y * solution(x, 0.0)
    
    return A, rhs

Exact solution vector

In [8]:
def exact_vector(N):
    h = 1.0 / N
    u_ex = np.zeros((N-1)**2)
    k = 0
    for j in range(1, N):
        for i in range(1, N):
            x, y = i*h, j*h
            u_ex[k] = solution(x, y)
            k += 1
    return u_ex


Conjugate gradient method

In [9]:
def conjugate_gradient(A: np.ndarray, b_vec: np.ndarray, eps: float, max_it: int) -> tuple[np.ndarray, int, np.ndarray]:
    it = 0
    norms_rk = []

    x_cur = np.zeros_like(b_vec)
    r_cur = b_vec - A @ x_cur
    p_cur = copy.deepcopy(r_cur)

    while np.linalg.norm(r_cur) > eps and it < max_it:
        r_prev = copy.deepcopy(r_cur)
        p_prev = copy.deepcopy(p_cur)
        alpha_prev = np.dot(r_prev, r_prev) / np.dot(A @ p_prev, p_prev)
        x_prev = copy.deepcopy(x_cur)
        x_cur = x_prev + alpha_prev * p_prev
        r_cur = r_prev - alpha_prev * A @ p_prev
        beta_prev = np.dot(r_cur, r_cur) / np.dot(r_prev, r_prev)
        p_cur = r_cur + beta_prev * p_prev
        # print(f"norm(r_cur) = {np.linalg.norm(r_cur)}\n")
        it += 1
        norms_rk.append(np.linalg.norm(r_cur))
    return x_cur, it, norms_rk

Richardson method

In [35]:
def Richardson(A: np.ndarray, b_vec: np.ndarray, eps: float, max_it: int) -> tuple[np.ndarray, int, np.ndarray]:
    it = 0
    norms_rk = []

    eigvals = np.linalg.eigvals(A)
    lambda_max = max(np.abs(eigvals))
    lambda_min = min(np.abs(eigvals))
    
    tau = 2.0 / (lambda_max + lambda_min)

    x = np.zeros_like(b_vec)
    r = A @ x - b_vec
    while np.linalg.norm(r) > eps and it < max_it:
        x = x - tau * r
        r = A @ x - b_vec
        # print(f"norm(r) = {np.linalg.norm(r)}\n")
        it += 1
        norms_rk.append(np.linalg.norm(r))

    return x, it, norms_rk

In [65]:
def Richardson_Chebyshev(A: np.ndarray, b_vec: np.ndarray, eps: float, max_it: int) -> tuple[np.ndarray, int, np.ndarray]:
    it = 0
    norms_rk = []

    eigvals = np.linalg.eigvals(A)
    lambda_max = np.max(eigvals.real)
    lambda_min = np.min(eigvals.real)
    # print(f"lambda_max = {lambda_max}")
    # print(f"lambda_min = {lambda_min}")

    c = (lambda_max + lambda_min) / 2.0
    d = (lambda_max - lambda_min) / 2.0

    x = np.zeros_like(b_vec)
    r = b_vec - A @ x

    while np.linalg.norm(r) > eps and it < max_it:
        mu_k = c + d * math.cos(math.pi * (2*it + 1) / (2*max_it))
        tau_cur = 1.0 / mu_k

        x = x + tau_cur * r
        r = b_vec - A @ x
        it += 1
        norms_rk.append(np.linalg.norm(r))

        # print(f"iter {it:3d}, ||r|| = {np.linalg.norm(r):.3e}")

    return x, it, norms_rk


Test

In [ ]:
# N = 54
# A, b_vec = SLAE_compilation(N)

# x_cg, it_cg, r_cg = conjugate_gradient(A, b_vec, eps=1e-9, max_it=10000)
# x_rich, it_rich, r_rich = Richardson(A, b_vec, eps=1e-9, max_it = 10000)
# x_rich_cheb, it_rich_cheb, r_rich_cheb = Richardson_Chebyshev(A, b_vec, eps=1e-9, max_it=10000)

# it_cg, it_rich, it_rich_cheb

Numerical experiments

In [10]:
number_of_nodes = np.arange(4, 30, 5)

1) Error on iterations number

In [ ]:
max_iterations_numbers = np.arange(1, 101, 10)

eps = 1e-10

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel() 

for index, N in enumerate(number_of_nodes):
    A, b_vec = SLAE_compilation(N)
    errors_cg = []
    errors_rich = []
    numbers_of_iterations = []

    u_ex = exact_vector(N)

    for max_it in max_iterations_numbers:
        u_cg, it_cg, _ = conjugate_gradient(A=A, b_vec=b_vec, eps=eps, max_it=max_it)
        u_rich, it_rich, _ = Richardson(A=A, b_vec=b_vec, eps=eps, max_it=max_it)
        errors_cg.append(np.max(np.abs(u_cg - u_ex)))
        errors_rich.append(np.max(np.abs(u_rich - u_ex)))

    ax = axes[index]
    ax.semilogy(max_iterations_numbers, errors_cg, color = 'blue', marker="o", label="CG")
    ax.semilogy(max_iterations_numbers, errors_rich, color = 'green', marker="s", label="Richardson")
    ax.set_title(f"N={N}")
    ax.set_xlabel("number of iterations")
    ax.set_ylabel("||u - u_ex||")
    ax.legend()
    ax.grid(True)

fig.suptitle("Accuracy on iterations number for different grids", fontsize=18)
plt.tight_layout()
plt.savefig("Accuracy_on_iterations_number.pdf")
plt.show()

2. Stability

In [12]:
def perturb_b_vec(b_vec, eps):
    rng = np.random.default_rng()
    delta = rng.normal(size=b_vec.shape)
    delta = delta / np.linalg.norm(delta) * (eps * np.linalg.norm(b_vec))
    return b_vec + delta

def perturb_A(A, eps):
    rng = np.random.default_rng()
    E = rng.normal(size=A.shape)
    normA = np.linalg.norm(A, ord='fro')
    E = E / np.linalg.norm(E, ord='fro') * (eps * normA)
    return A + E

In [13]:
max_it = 1e10

Matrix disturbance

In [ ]:
N = 29
eps_values = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
iters_cg, iters_rich = [], []
errors_cg, errors_rich = [], []
A, b_vec = SLAE_compilation(N)
x_cg, it_cg, _ = conjugate_gradient(A=A, b_vec=b_vec, eps=eps, max_it=max_it)
x_rich, it_rich, _ = Richardson(A=A, b_vec=b_vec, eps=eps, max_it=max_it)
for eps_value in eps_values:
    A_pert = perturb_A(A=A, eps=eps_value)
    x_cg_p, it_cg_p, _ = conjugate_gradient(A=A_pert, b_vec=b_vec, eps=1e-10, max_it=max_it)
    x_rich_p, it_rich_p, _ = Richardson(A=A_pert, b_vec=b_vec, eps=eps, max_it=max_it)
    error_solution_cg = np.linalg.norm(x_cg_p - x_cg)
    error_solution_rich = np.linalg.norm(x_rich_p - x_rich)
    iters_cg.append(it_cg_p)
    errors_cg.append(error_solution_cg)
    errors_rich.append(error_solution_rich)
    # print(f"eps = {eps_value}\n")
    # print(f"||x_p = x|| = {error_solution_cg}\n")
    # print(f"Non-perturbated: {it_cg} iterations, perturbated: {it_cg_p} iterations \n\n")

plt.loglog(eps_values, errors_cg, marker = "o", color="blue")
plt.loglog(eps_values, errors_rich, marker = "o", color = "green")
plt.xlabel("Disturbance amplitude ε")
plt.ylabel("||x_pert - x||")
plt.title(f"Solution stability (both), {N + 1} nodes")
plt.grid(True, which="both")
plt.savefig("Solution_stability.pdf")
plt.show()

In [15]:
eps_values = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]

In [ ]:
fig = plt.figure(figsize=(12, 24))
outer = fig.add_gridspec(6, 1) 

for ind, N in enumerate(number_of_nodes):

    inner = outer[ind].subgridspec(1, 2, wspace=0.3)

    ax_cg = fig.add_subplot(inner[0])
    ax_rich = fig.add_subplot(inner[1])

    iters_cg = []
    errors_cg = []
    iters_rich = []
    errors_rich = []

    A, b_vec = SLAE_compilation(N)
    res_cg, it_cg, _ = conjugate_gradient(A=A, b_vec=b_vec, eps=eps, max_it=max_it)
    res_rich, it_rich, _ = Richardson(A=A, b_vec=b_vec, eps=eps, max_it=max_it)

    for eps_value in eps_values:
        A_pert = perturb_A(A=A, eps=eps_value)
        x_cg_p, it_cg_p, _ = conjugate_gradient(A=A_pert, b_vec=b_vec, eps=1e-10, max_it=max_it)
        x_rich_p, it_rich_p, _ = Richardson(A=A_pert, b_vec=b_vec, eps=1e-10, max_it=max_it)
        
        error_solution_cg = np.linalg.norm(x_cg_p - res_cg)
        error_solution_rich = np.linalg.norm(x_rich_p - res_rich)

        iters_cg.append(it_cg_p)
        errors_cg.append(error_solution_cg)
        iters_rich.append(it_rich_p)
        errors_rich.append(error_solution_rich)


    ax_cg.semilogx(eps_values, iters_cg, marker="o", color = "blue")
    ax_cg.set_xlabel("Disturbance amplitude ε")
    ax_cg.set_ylabel("Number of iterations")
    ax_cg.set_title(f"CG, {N + 1} nodes")
    ax_cg.grid(True)


    ax_rich.semilogx(eps_values, iters_rich, marker="o", color="green")
    ax_rich.set_xlabel("Disturbance amplitude ε")
    ax_rich.set_ylabel("Number of iterations")
    ax_rich.set_title(f"Richardson, {N + 1} nodes")
    ax_rich.grid(True)

fig.suptitle("Number of iterations on disturbance amplitude", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("Iterations_on_matrix_disturbance.pdf")
plt.show()



Right part vector disturbance

In [ ]:
N = 29
eps_cur = 1e-2
A, b_vec = SLAE_compilation(N)
b_vec_pert = perturb_b_vec(b_vec=b_vec, eps=eps_cur)

x_cg, it_cg, _ = conjugate_gradient(A=A, b_vec=b_vec, eps=eps, max_it=max_it)
x_cg_pert, it_cg_pert, _ = conjugate_gradient(A=A, b_vec=b_vec_pert, eps=eps, max_it=max_it)
x_rich, it_rich, _ = Richardson(A=A, b_vec=b_vec, eps=eps, max_it=max_it)
x_rich_pert, it_rich_pert, _ = Richardson(A=A, b_vec=b_vec_pert, eps=eps, max_it=max_it)

error_solution_cg = np.linalg.norm(x_cg - x_cg_pert)
error_solution_rich = np.linalg.norm(x_rich - x_rich_pert)

print(f"{N + 1} nodes, eps = {eps_cur}\n")
print("Conjugate gradient:\n")
print(f"||x_p - x|| = {error_solution_cg}\n")
print(f"Non-perturbated: {it_cg} iterations, perturbated: {it_cg_pert} iterations \n\n")
print("Richardson:\n")
print(f"||x_rich - x|| = {error_solution_rich}\n")
print(f"Non-perturbated: {it_rich} iterations, perturbated: {it_rich_pert} iterations \n\n")


3. Rate of convergence

The abscissa axis is the iteration number. The ordinate axis is the norm of the current direction vector (the stopping criteria).

In [ ]:
eps = 1e-5

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

for ind, N in enumerate(number_of_nodes):
    A, b_vec = SLAE_compilation(N)
    x_cg, it_cg, rk_cg = conjugate_gradient(A=A, b_vec=b_vec, eps=eps, max_it=max_it)
    x_rich, it_rich, rk_rich = Richardson(A=A, b_vec=b_vec, eps=eps, max_it=max_it)

    iters_cg = np.arange(0, it_cg)
    iters_rich = np.arange(0, it_rich)

    ax = axes[ind]
    ax.semilogy(iters_cg, rk_cg, label="CG", color = "blue")
    ax.semilogy(iters_rich, rk_rich, label="Richardson", color = "green")
    ax.axhline(y = 1e-5, color = "red")
    ax.grid(True)
    ax.set_title(f"N = {N + 1}")
    ax.set_xlabel("Iteration number")
    ax.set_ylabel("Residual norm")
    ax.legend()

fig.suptitle(r"Residual on iteration number, $\varepsilon = 10^{-5}$", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("Residual_on_iteration_number.pdf")
plt.show()


Computetion time on number of nodes

In [19]:
import time

In [ ]:
N_values = np.arange(4, 55, 5)    
times_cg = []            
times_rich = []                     

for N_value in N_values:
    A, b_vec = SLAE_compilation(N_value)

    start = time.perf_counter()
    x, it, _ = conjugate_gradient(A=A, b_vec=b_vec, eps=1e-8, max_it=10000)
    times_cg.append(time.perf_counter() - start)

    start = time.perf_counter()
    x2, it2, _ = Richardson(A=A, b_vec=b_vec, eps=1e-8, max_it=10000)
    times_rich.append(time.perf_counter() - start)


plt.figure(figsize=(7,5))
plt.semilogy(N_values + 1, times_cg, color='blue', label='CG', marker='o')
plt.semilogy(N_values + 1, times_rich, color='green', label='Richardson', marker='o')
plt.xlabel("N")
plt.ylabel("time, s")
plt.legend()
plt.grid(True, which='both')
plt.title("Computation time on number of nodes")
plt.savefig("Computation time on number of nodes.pdf")
plt.tight_layout()
plt.show()